In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from datetime import date, timedelta

In [2]:
# get historical data for all tickers per date

def get_data_for_date(db, d):

    conn = sqlite3.connect(db)
    c = conn.cursor()
    delta = timedelta(days=1)
    dt = date.today() - delta * d
    c.execute('select count(*) from quotes where dt = ?', (dt.isoformat(),))

    if c.fetchone()[0] > 0:
        conn.close()
        return

    url = 'https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json?date='
    start = 0

    while True:
        r = requests.get(url + dt.isoformat() + '&start=' + str(start))
        data = r.json()['history']['data']
        page = r.json()['history.cursor']['data'][0]
        arr = []

        for d in data:
            arr.append((d[1], d[2], d[3], d[6], d[7], d[8], d[11], d[12]))

        c.executemany('''
        INSERT INTO quotes (dt, name, ticker, open, low, high, close, vol) 
        VALUES (?,?,?,?,?,?,?,?)
        ''', arr)
        start += 100

        if start > page[1]:
            break

    conn.commit()
    conn.close()

In [3]:
# get absent data and load into 'trade.db'

con = sqlite3.connect('trade.db')
c = con.cursor()
max_dt = c.execute('select max(dt) from quotes').fetchone()[0]
print('Before update: ', max_dt)
days = (date.today() - date.fromisoformat(max_dt)).days

for i in range(0, days + 1):
    get_data_for_date('trade.db', i)

max_dt = c.execute('select max(dt) from quotes').fetchone()[0]
print('After update:  ', max_dt)
con.close()

Before update:  2020-07-28
After update:   2020-07-31


In [4]:
# get list of tickers for 5 years

con = sqlite3.connect('trade.db')
tickers = pd.read_sql_query('''
select ticker 
from quotes 
where close is not null
group by ticker having min(dt) < date('now', '-5 year') and max(dt) >= date('now', '-3 day')
''', con).ticker.values

con.close()
len(tickers)

222

In [5]:
# load dataset for last 5 years of single ticker 

def load(ticker):
    con = sqlite3.connect('trade.db')
    data = pd.read_sql_query('''
    select * from quotes where close is not null
    and ticker = "{}" and dt between date('now', '-5 year') and date('now')
    order by dt
    '''.format(ticker), con)
    con.close()
    return data

In [6]:
# adding RSI in dataset

def rsi(data, period):
    data['ch'] = data.close.diff()
    data['plus'] = data.ch[data.ch >= 0]
    data.plus.fillna(value=0, inplace=True)
    data.loc[period, 'plus'] = data.plus[1:period+1].mean()
    data['plus_m'] = data.plus[period:].ewm(com=period-1, adjust=False).mean()
    data['neg'] = abs(data.ch[data.ch <= 0])
    data.neg.fillna(value=0, inplace=True)
    data.loc[period, 'neg'] = data.neg[1:period+1].mean()
    data['neg_m'] = data.neg[period:].ewm(com=period-1, adjust=False).mean()
    data['rsi'] = 100 - (100 / (1 + (data.plus_m / data.neg_m)))
    data.drop(['ch', 'plus', 'plus_m', 'neg', 'neg_m'], axis=1, inplace=True)
    return data

In [7]:
# adding MACD in dataset

def macd(data, fast=12, slow=26, signal=9):
    data['fast'] = data.close.ewm(span=fast, adjust=False).mean()
    data['slow'] = data.close.ewm(span=slow, adjust=False).mean()
    data['macd'] = data.fast - data.slow
    data['signal'] = data.macd.ewm(span=signal, adjust=False).mean()
    data['macd'] = data.macd - data.signal
    data.drop(['fast', 'slow', 'signal'], axis=1, inplace=True)
    return data

In [8]:
# adding bollinger in dataset

def bollinger(data, period):
    data['sma'] = data.close.rolling(window=period).mean()
    data['dev'] = data.close - data.sma
    data['sigma'] = data.close.rolling(window=20).std(ddof=0)
    data['upper'] = data.sma + data.sigma * 2
    data['lower'] = data.sma - data.sigma * 2
    data.drop(['sigma'], axis=1)
    return data

In [9]:
# SMA-based strategy

def strategy_sma(data, period):
    deal = []
    deals = []
    ingame = False

    for i in range(period + 1, len(data) - 1):
        if ingame is False and data.dev[i-2] < 0 and data.dev[i-1] >= 0 and data.rsi[i-1] < 70:
            deal = [data.ticker[i], data.dt[i], data.open[i], data.rsi[i-1]]
            ingame = True
        if ingame is True and (data.dev[i-1] < 0 or data.rsi[i-1] > 80):
            deal.extend([data.dt[i], data.open[i], data.rsi[i-1]])
            deals.append(deal)
            deal = []
            ingame = False
    
    return pd.DataFrame(deals, columns=['ticker', 'sdate', 'sprice', 'srsi', 'edate', 'eprice', 'ersi'])    

In [10]:
# MACD-based strategy

def strategy_macd(data, period=26):
    deal = []
    deals = []
    ingame = False

    for i in range(period + 1, len(data) - 1):
        if ingame is False and data.macd[i-2] < 0 and data.macd[i-1] >= 0:
            deal = [data.ticker[i], data.dt[i], data.open[i], data.rsi[i-1]]
            ingame = True
        if ingame is True and data.macd[i-1] < 0:
            deal.extend([data.dt[i], data.open[i], data.rsi[i-1]])
            deals.append(deal)
            deal = []
            ingame = False
    
    return pd.DataFrame(deals, columns=['ticker', 'sdate', 'sprice', 'srsi', 'edate', 'eprice', 'ersi'])    

In [11]:
# main function for trading

def run(t, period, cash):
    
    data = load(t)
    data = bollinger(data, period)
    data = rsi(data, period)    
    data = macd(data)
    
    check = 0
    
#     if data.dev[len(data)-1] >= 0 and data.dev[len(data)-2] < 0:
#         check = 1

#     deals = strategy_sma(data, period)

    if data.macd[len(data)-1] >= 0 and data.macd[len(data)-2] < 0:
        check = 1
    
    deals = strategy_macd(data)
    deals['profit'] = (deals.eprice - deals.sprice) / deals.sprice
    deals['total'] = deals.profit + 1
    
    win = len(deals[deals.profit > 0])
    loss = len(deals[deals.profit <= 0])
    percent_of_win = round(len(deals[deals.profit > 0]) * 100 / len(deals), 2)
    sharp = round(deals.profit.mean() / deals.profit.std(), 2)

    worse = round(min(deals.profit * cash), 2)
    best = round(max(deals.profit * cash), 2)
    total = round(sum(deals.profit * cash - 6 - (deals.profit + 1) * 6) + 10000, 2)

    res = cash
    drawdown = np.inf

    for p in deals.profit:
        res += res * p
        if drawdown > res:
            drawdown = res
    
    return (t, period, win, loss, percent_of_win, sharp, \
           worse, best, total, round(drawdown, 2), round(res, 2), check), deals

In [12]:
# fill an array with results of trading for each ticker

stat = []
deals = []

for t in tickers:
    for i in range(20, 21):
        a, b = run(t, i, 10000)
        stat.append(a)
        deals.append(b)

data = pd.DataFrame(stat)
data.columns = ['quote', 'per', 'win', 'loss', '% of win', 'sharp', 
                'worse', 'best', 'total', 'drawdown', 'cumsum', 'check']
deals = pd.concat(deals, axis=0)

In [20]:
data.describe()[1:2]

,per,win,loss,% of win,sharp,worse,best,total,drawdown,cumsum,check
mean,20.0,16.608108,27.175676,38.096622,0.100766,-1597.824505,5833.002252,15974.033874,7528.600586,17177.083018,0.054054


In [21]:
data[data.check==1].sort_values(by=['sharp'], ascending=False)

,quote,per,win,loss,% of win,sharp,worse,best,total,drawdown,cumsum,check
188,TRMK,20,19,18,51.35,0.31,-701.75,3172.41,18792.59,10156.86,22463.31,1
179,TGKA,20,20,23,46.51,0.28,-713.74,4990.37,23101.38,9279.25,30999.26,1
58,KCHEP,20,15,21,41.67,0.22,-2078.85,11180.56,30718.30,7425.76,34968.58,1
137,QIWI,20,16,25,39.02,0.18,-912.66,3138.83,16596.02,8553.42,17032.18,1
159,RUSP,20,16,25,39.02,0.18,-1574.64,3836.21,17012.59,10281.99,17435.08,1
219,YRSBP,20,7,11,38.89,0.16,-2478.63,9833.33,17285.19,6404.97,13736.89,1
5,ALBK,20,10,12,45.45,0.12,-4437.22,23000.00,23732.70,7222.22,10587.35,1
209,VSYDP,20,10,14,41.67,0.12,-2000.00,8560.00,16751.08,5643.52,12528.16,1
34,ENRU,20,21,25,45.65,0.11,-1182.50,3323.08,13157.92,8296.09,12949.28,1
29,DIOD,20,16,36,30.77,0.07,-1121.50,2409.64,11860.32,8272.51,11401.36,1


In [22]:
data.sort_values(by=['sharp'], ascending=False)

,quote,per,win,loss,% of win,sharp,worse,best,total,drawdown,cumsum,check
165,SBER,20,20,17,54.05,0.48,-591.76,1829.69,20059.79,11464.56,26599.22,0
76,KZOS,20,26,20,56.52,0.44,-600.00,1831.28,19113.96,11238.78,24829.31,0
36,FESH,20,19,17,52.78,0.42,-1293.53,6197.40,34636.82,9038.36,78970.13,0
207,VSMO,20,24,16,60.00,0.38,-576.92,1674.36,16626.47,10043.03,19423.33,0
21,CHEP,20,23,14,62.16,0.38,-1828.36,8993.29,34704.53,9167.95,75277.47,0
70,KRSB,20,26,19,57.78,0.37,-973.15,4692.74,23332.19,8931.49,34716.32,0
166,SBERP,20,20,22,47.62,0.36,-756.52,2258.91,19188.72,10707.99,24103.16,0
103,MRKS,20,21,24,46.67,0.35,-1123.19,5712.92,34720.53,8684.83,75207.92,0
97,MISB,20,10,11,47.62,0.35,-1102.36,10650.00,29853.18,10774.06,43057.43,0
119,NKNCP,20,22,18,55.00,0.34,-1171.88,8024.69,30497.54,9054.05,55676.87,0
